In [60]:
import pandas as pd
import numpy as np

In [61]:
df = pd.read_csv('scada_logs.csv', sep=';')
df.head(5)

,Дата,Время,Сообщение,Класс тревоги
0,12.11.2025,07:00:07,4.2.9 Конвейер - остановлен,_Info
1,12.11.2025,06:59:58,4.1.2 Нория - Ток: рабочий сброшен,_Info
2,12.11.2025,06:59:58,4.1.2 Нория - Ток: холостой ход активен,_Info
3,12.11.2025,06:59:58,4.1.2 Нория - остановлен,_Info
4,12.11.2025,06:59:47,4.2.6 Конвейер - остановлен,_Info


In [62]:
# Меняем структуру
df['datetime'] = pd.to_datetime(df['Дата'] + ' ' + df['Время'], dayfirst=True)
scada_logs = df[['datetime', 'Сообщение', 'Класс тревоги']].copy()
scada_logs.columns = ['datetime', 'message', 'alarm_class']

# Округляем до следующего часа если минуты >= 45, иначе оставляем текущий час
scada_logs['hour'] = scada_logs['datetime'].apply(
    lambda x: (x + pd.Timedelta(hours=1)).hour if x.minute >= 45 else x.hour)

scada_logs.info()
scada_logs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154598 entries, 0 to 154597
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   datetime     154594 non-null  datetime64[ns]
 1   message      154594 non-null  object        
 2   alarm_class  154592 non-null  object        
 3   hour         154594 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 4.7+ MB


,datetime,message,alarm_class,hour
0,2025-11-12 07:00:07,4.2.9 Конвейер - остановлен,_Info,7.0
1,2025-11-12 06:59:58,4.1.2 Нория - Ток: рабочий сброшен,_Info,7.0
2,2025-11-12 06:59:58,4.1.2 Нория - Ток: холостой ход активен,_Info,7.0
3,2025-11-12 06:59:58,4.1.2 Нория - остановлен,_Info,7.0
4,2025-11-12 06:59:47,4.2.6 Конвейер - остановлен,_Info,7.0


In [63]:
scada_logs['alarm_class'].unique()


array(['_Info', '_Warnings', '_Errors', nan], dtype=object)

In [64]:
errors_df = scada_logs[scada_logs['alarm_class'] == '_Errors'].copy()
print(f"Записей с _Errors: {len(errors_df)}")


errors_df.head()

Записей с _Errors: 789


,datetime,message,alarm_class,hour
1346,2025-11-11 11:10:37,1.1.3 Нория - авария сброшена,_Errors,11.0
1347,2025-11-11 11:10:01,1.1.3 Нория - авария активна,_Errors,11.0
1349,2025-11-11 11:10:00,1.1.3 Нория - сработал датчик скорости,_Errors,11.0
1351,2025-11-11 11:09:49,1.1.3 Нория - авария сброшена,_Errors,11.0
1353,2025-11-11 11:09:18,1.1.3 Нория - авария активна,_Errors,11.0


In [75]:
errors_df[['equipment_number', 'event_description']] = errors_df['message'].str.split(' - ', n=1, expand=True)
desired_order = [
    'datetime', 'hour', 'message', 'equipment_number', 'event_description', 
    'alarm_class'
]

# Переупорядочиваем колонки
errors_df = errors_df[desired_order]
errors_df.head()

,datetime,hour,message,equipment_number,event_description,alarm_class
1346,2025-11-11 11:10:37,11.0,1.1.3 Нория - авария сброшена,1.1.3 Нория,авария сброшена,_Errors
1347,2025-11-11 11:10:01,11.0,1.1.3 Нория - авария активна,1.1.3 Нория,авария активна,_Errors
1349,2025-11-11 11:10:00,11.0,1.1.3 Нория - сработал датчик скорости,1.1.3 Нория,сработал датчик скорости,_Errors
1351,2025-11-11 11:09:49,11.0,1.1.3 Нория - авария сброшена,1.1.3 Нория,авария сброшена,_Errors
1353,2025-11-11 11:09:18,11.0,1.1.3 Нория - авария активна,1.1.3 Нория,авария активна,_Errors


In [76]:
errors_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 789 entries, 1346 to 151082
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           789 non-null    datetime64[ns]
 1   hour               789 non-null    float64       
 2   message            789 non-null    object        
 3   equipment_number   789 non-null    object        
 4   event_description  789 non-null    object        
 5   alarm_class        789 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 43.1+ KB


In [79]:
sensor_events = errors_df[errors_df['event_description'].str.contains('датчик', case=False, na=False)]
sensor_events.info()
sensor_events.head()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 1349 to 145418
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           107 non-null    datetime64[ns]
 1   hour               107 non-null    float64       
 2   message            107 non-null    object        
 3   equipment_number   107 non-null    object        
 4   event_description  107 non-null    object        
 5   alarm_class        107 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 5.9+ KB


,datetime,hour,message,equipment_number,event_description,alarm_class
1349,2025-11-11 11:10:00,11.0,1.1.3 Нория - сработал датчик скорости,1.1.3 Нория,сработал датчик скорости,_Errors
1354,2025-11-11 11:09:18,11.0,1.1.3 Нория - сработал датчик скорости,1.1.3 Нория,сработал датчик скорости,_Errors
1593,2025-11-11 10:47:20,11.0,2.2.2 Конвейер - сработал датчик натяжен...,2.2.2 Конвейер,сработал датчик натяжения цепи,_Errors
1713,2025-11-11 10:08:26,10.0,2.2.2 Конвейер - сработал датчик натяжен...,2.2.2 Конвейер,сработал датчик натяжения цепи,_Errors
5887,2025-11-11 22:48:16,23.0,5.3.5 Конвейер - сработал датчик скорости,5.3.5 Конвейер,сработал датчик скорости,_Errors


# Сохраняем DataFrame с событиями аварий датчиков

In [81]:
output_file = 'sensor_events.csv'
sensor_events.to_csv(output_file, sep=';', index=False, encoding='utf-8-sig')